In [1]:
import sys
sys.path.insert(0,'../backend')

import numpy as np

import webotsgym as wg
from stable_baselines import PPO1

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
config = wg.WbtConfig()
config.world_size = 8
config.num_obstacles = 16
config.sim_mode = wg.config.SimSpeedMode.RUN

env = wg.WbtGymGrid(train=True, config=config)

Accepting on Port:  10201


In [4]:
# model loading
model_name = "PPO_lam3+vs_500k"
model = PPO1.load("model/grid/{}".format(model_name))

Loading a model without an environment, this model cannot be trained until it has a valid environment.




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
obs = env.reset(128)
for i in range(200):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)

    if env.get_target_distance() < 0.1:
        print(i + 1)
        break

KeyboardInterrupt: 